In [0]:
%sql
USE f1_demo;

In [0]:
drivers_day1_df = spark.read \
.option("inferSchema", True) \
.json("dbfs:/FileStore/raw/2021-03-28/drivers.json") \
.filter("driverId <= 10") \
.select("driverId", "dob", "name.forename", "name.surname")

In [0]:
display(drivers_day1_df)


In [0]:
drivers_day1_df.createOrReplaceTempView("drivers_day1")

In [0]:
from pyspark.sql.functions import upper

drivers_day2_df = spark.read \
.option("inferSchema", True) \
.json("dbfs:/FileStore/raw/2021-03-28/drivers.json") \
.filter("driverId BETWEEN 6 AND 15") \
.select("driverId", "dob", upper("name.forename").alias("forename"), upper("name.surname").alias("surname"))

display(drivers_day2_df)

In [0]:
drivers_day2_df.createOrReplaceTempView("drivers_day2")

In [0]:
from pyspark.sql.functions import upper

drivers_day3_df = spark.read \
.option("inferSchema", True) \
.json("dbfs:/FileStore/raw/2021-03-28/drivers.json") \
.filter("driverId BETWEEN 1 AND 5 OR driverId BETWEEN 16 AND 20") \
.select("driverId", "dob", upper("name.forename").alias("forename"), upper("name.surname").alias("surname"))

In [0]:
%sql
CREATE TABLE IF NOT EXISTS f1_demo.drivers_merge (
driverId INT,
dob DATE,
forename STRING, 
surname STRING,
createdDate DATE, 
updatedDate DATE
)
USING DELTA

- Applying Merge operation

In [0]:
%sql
MERGE INTO f1_demo.drivers_merge tgt
USING drivers_day1 upd
ON tgt.driverId = upd.driverId
WHEN MATCHED THEN
  UPDATE SET tgt.dob = upd.dob,
             tgt.forename = upd.forename,
             tgt.surname = upd.surname,
             tgt.updatedDate = current_timestamp
WHEN NOT MATCHED
  THEN INSERT (driverId, dob, forename,surname,createdDate ) VALUES (driverId, dob, forename,surname, current_timestamp)

In [0]:
%sql SELECT * FROM f1_demo.drivers_merge;

merging with second data

In [0]:
%sql
MERGE INTO f1_demo.drivers_merge tgt
USING drivers_day2 upd
ON tgt.driverId = upd.driverId
WHEN MATCHED THEN
  UPDATE SET tgt.dob = upd.dob,
             tgt.forename = upd.forename,
             tgt.surname = upd.surname,
             tgt.updatedDate = current_timestamp
WHEN NOT MATCHED
  THEN INSERT (driverId, dob, forename,surname,createdDate ) VALUES (driverId, dob, forename,surname, current_timestamp)

In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge;
  